In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langchain.tools import tool 
import os
from langchain.agents.middleware import wrap_tool_call, dynamic_prompt, ModelRequest
from langchain_core.messages import ToolMessage
from pydantic import BaseModel, Field
from typing import Optional
import pandas as pd
from search_food import RAGSystem, Text
#from my_tools import get_recipes_by_ingredients


In [3]:
txt = Text("saved_txt.txt")
rag = RAGSystem(txt)

Векторная БД загружена из файла


In [4]:

kk = pd.read_excel("kk.xlsx")

def parse_range(value):
    """Преобразует строку с диапазоном в среднее число"""
    if pd.isna(value):
        return None
    try:
        # Заменяем запятые на точки и разделяем по дефису
        parts = str(value).replace(',', '.').split('-')
        # Убираем пробелы и преобразуем в числа
        numbers = [float(part.strip()) for part in parts]
        # Возвращаем среднее значение
        return sum(numbers) / len(numbers)
    except:
        return None

# Применяем ко всем столбцам с диапазонами
kk['squirrels_avg'] = kk['squirrels'].apply(parse_range)
kk['fats_avg'] = kk['fats'].apply(parse_range)
kk['carbohydrates_avg'] = kk['carbohydrates'].apply(parse_range)
kk['callories_avg'] = kk['callories'].apply(parse_range)
kk

,Product,squirrels,fats,carbohydrates,callories,squirrels_avg,fats_avg,carbohydrates_avg,callories_avg
0,Хлеб пшеничный из муки высшего сорта,"7,5-7,6","0,8-2,9","49,2-51,4",235-262,7.55,1.85,50.3,248.5
1,Хлеб пшеничный из муки 1 сорта,7.6,0.9,49.7,226,7.60,0.90,49.7,226.0
2,Хлеб из ржано-пшеничной муки,6.8,1.2,46.4,215,6.80,1.20,46.4,215.0
3,Хлеб ржаной,4.7,0.7,49.8,214,4.70,0.70,49.8,214.0
4,"Хлеб ""Бородинский""",6.9,1.3,40.9,208,6.90,1.30,40.9,208.0
...,...,...,...,...,...,...,...,...,...
631,Саке,0.5,-,5,134,0.50,NaN,5.0,134.0
632,Самогон,0.1,0.1,0.4,235,0.10,0.10,0.4,235.0
633,Спирт этиловый 96%,4,-,3.8,710,4.00,NaN,3.8,710.0
634,Текила,1.4,0.3,24,231,1.40,0.30,24.0,231.0


In [5]:
kk[kk["Product"] == 'Сыр "Гауда"']

,Product,squirrels,fats,carbohydrates,callories,squirrels_avg,fats_avg,carbohydrates_avg,callories_avg
96,"Сыр ""Гауда""",25,27,2,356,25.0,27.0,2.0,356.0


In [6]:
class ProductSearch(BaseModel):
    """Ввод данных для запросов о еде"""

    name: Optional[str] = Field(description="Название продукта", default=None)
    squirrels: Optional[str] = Field(description="Белки", default=None)
    fats: Optional[str] = Field(description="Жиры", default=None)
    carbohydrates: Optional[str] = Field(description="Углеводы", default=None)
    callories: Optional[str] = Field(description="Энергия", default=None)
    squirrels_avg: Optional[str] = Field(description="Белки среднее значение", default=None)
    fats_avg: Optional[str] = Field(description="Жиры среднее значение", default=None)
    carbohydrates_avg: Optional[str] = Field(description="Углеводы среднее значение", default=None)
    callories_avg: Optional[str] = Field(description="Энергия среднее значение", default=None)
    
    # Добавляем параметры для диапазонного поиска
    min_squirrels: Optional[int] = Field(description="Минимальное количество белков", default=None)
    max_squirrels: Optional[int] = Field(description="Максимальное количество белков", default=None)
    min_fats: Optional[int] = Field(description="Минимальное количество жиров", default=None)
    max_fats: Optional[int] = Field(description="Максимальное количество жиров", default=None)
    min_carbohydrates: Optional[int] = Field(description="Минимальное количество углеводов", default=None)
    max_carbohydrates: Optional[int] = Field(description="Максимальное количество углеводов", default=None)
    min_callories: Optional[int] = Field(description="Минимальная энергетическая ценность", default=None)
    max_callories: Optional[int] = Field(description="Максимальная энергетическая ценность", default=None)
    number_products: Optional[int] = Field(description="Количество продуктов", default=None)
    
    sort_order: Optional[str] = Field(
        description="Порядок сортировки (наибольшие белки, наименьшие белки, наибольшие жиры, наименьшие жиры, наибольшие углеводы, наименьшие углеводы,наибольшие каллории, наименьшие каллории)",
        default=None,
    )

In [7]:
@tool(args_schema=ProductSearch)
def search_food(
    name: Optional[str] = None,
    squirrels: Optional[str] = None,
    fats: Optional[str] = None,
    carbohydrates: Optional[str] = None,
    callories: Optional[str] = None,
    squirrels_avg: Optional[str] = None,
    fats_avg: Optional[str] = None,
    carbohydrates_avg: Optional[str] = None,
    callories_avg: Optional[str] = None,
    min_squirrels: Optional[int] = None,
    max_squirrels: Optional[int] = None,
    min_fats: Optional[int] = None,
    max_fats: Optional[int] = None,
    min_carbohydrates: Optional[int] = None,
    max_carbohydrates: Optional[int] = None,
    min_callories: Optional[int] = None,
    max_callories: Optional[int] = None,
    sort_order: Optional[str] = None,
    number_products: Optional[int] = None
) -> str:
    """Поиск продуктов питания по различным критериям питания"""
    
    x = kk.copy()
    
    # Apply filters
    if name:  
        x = x[x["Product"].apply(lambda x: name.lower() in str(x).lower())]
    if min_callories: 
        x = x[x["callories_avg"] >= min_callories]
    if max_callories: 
        x = x[x["callories_avg"] <= max_callories]
    if min_carbohydrates: 
        x = x[x["carbohydrates_avg"] >= min_carbohydrates]
    if max_carbohydrates: 
        x = x[x["carbohydrates_avg"] <= max_carbohydrates]
    if min_fats: 
        x = x[x["fats_avg"] >= min_fats]
    if max_fats: 
        x = x[x["fats_avg"] <= max_fats]
    if min_squirrels: 
        x = x[x["squirrels_avg"] >= min_squirrels]
    if max_squirrels: 
        x = x[x["squirrels_avg"] <= max_squirrels]
    if sort_order:
        if sort_order == "наибольшие белки":x = x.sort_values('squirrels_avg', ascending=False)
        if sort_order == "наименьшие белки":x = x.sort_values('squirrels_avg', ascending=True)
        if sort_order == "наибольшие жиры":x = x.sort_values('fats_avg', ascending=False)
        if sort_order == "наименьшие жиры":x = x.sort_values('fats_avg', ascending=True)
        if sort_order == "наибольшие углеводы":x = x.sort_values('carbohydrates_avg', ascending=False)
        if sort_order == "наименьшие углеводы":x = x.sort_values('carbohydrates_avg', ascending=True)
        if sort_order == "наибольшие каллории":x = x.sort_values('callories_avg', ascending=False)
        if sort_order == "наименьшие каллории":x = x.sort_values('callories_avg', ascending=True)
    if number_products: x = x.head(number_products)
    # Формируем результат
    result = "Результаты поиска продуктов:\n"
    
    for i, (idx, food) in enumerate(x.iterrows(), 1):
        result += f"{i}. {food['Product']} - Б: {food.get('squirrels_avg', 'N/A')}г, Ж: {food.get('fats_avg', 'N/A')}г, У: {food.get('carbohydrates_avg', 'N/A')}г, Кал: {food.get('callories_avg', 'N/A')}\n"
    
    result += f"\nНайдено продуктов: {len(x)}"
    
    return result

In [8]:
carts = {}

class AddToCart(BaseModel):
    """Ввод данных для добавления продукта в корзину"""
    name: str = Field(description="Точное название еды, чтобы положить в корзину", default=None)
    count: float = Field(description="Количество, которое нужно положить в корзину. Измеряется в граммах",default=100.0)

@tool(args_schema=AddToCart)
def add_product_to_cart(
    name: Optional[str] = None,
    count: Optional[float] = 100.0
    ):
    """Добавление продуктов в коризну carts"""
    
    if name not in kk['Product'].values:
        return f"Продукт '{name}' не найден. Пожалуйста, проверьте название."
    
    # Добавляем в корзину
    carts[name] = carts.get(name, 0) + count
    
    return f'{count} грамм "{name}" добавлено в корзину. Теперь в корзине: {carts[name]} грамм'


In [9]:
@tool(description="Показывает содержимое корзины")
def show_product_in_cart():
    """Показать продукты в корзине carts"""
    
    return carts

In [10]:
base_prompt = """Ты полезный ассистент по питанию. Ты получил результаты поиска продуктов. 
            Сформируй красивый, читаемый ответ для пользователя на русском языке.

            Если нужен список продуктов вот структурированный ответ:
            1. Краткое введение
            2. Группируй продукты по категориям (мясо, рыба, молочные и т.д.)
            3. Для каждого продукта укажи ключевые показатели
            4. Дай рекомендации

            Используй эмодзи и понятное форматирование."""

In [11]:
@tool
def search_dish(question: str) -> str:
    """Искать информацию о приготовлении блюда и продуктах для него"""
    answer = rag.ask_question(question)
    return answer

Еще инструменты
meal_plan - составить план питания на период

create_menu - создать меню для особого случая

calculate_nutrition - рассчитать пищевую ценность блюда

save_recipe - сохранить рецепт в избранное

share_recipe - поделиться рецептом
set_cooking_timer - установить таймер приготовления

convert_temperature - конвертировать температуры (C/F)

In [12]:
@tool(description="Удаляет один продукт из корзины")
def remove_from_cart(product_name: str, amount: Optional[float] = None):
    """Удалить продукт из корзины полностью или частично, если указаны граммы"""
    if amount:
        carts[product_name] -= amount
        if carts[product_name] <=0: del carts[product_name]
    else: del carts[product_name]
        

@tool(description="Удаляет абсолютно все продукты из корзины")
def clear_cart():
    """Удалить все продукты из корзины"""
    carts = {}

class Calculate(BaseModel):
    """Ввод данных для добавления продукта в корзину"""
    product_names: list[str] = Field(description="Список точный названий продуктов, из которых состоит блюдо", default=None)
    counts: list[int] = Field(description="Список количества продуктов, которое нужно положить в корзину. Измеряется в граммах",default=100.0)

@tool(args_schema=Calculate)
def calculate_all_products_nutrition(product_names: list[str],counts:list[int]):
    """Посчитать общую ценность всего блюда"""
    count = 0
    result = {}
    for i  in range(len(product_names)):
        name = product_names[i]
        weight = counts[i]
        
        try:
            product_nutrition = calculate_one_product_nutrition(name, weight)
            
            if isinstance(product_nutrition, str):
                return f"Ошибка для продукта '{name}': {product_nutrition}"
            
            result["squirrels_avg"] += product_nutrition["squirrels_avg"]
            result["fats_avg"] += product_nutrition["fats_avg"]
            result["carbohydrates_avg"] += product_nutrition["carbohydrates_avg"]
            result["callories_avg"] += product_nutrition["callories_avg"]
            total_count += weight
            
        except KeyError as e:
            return f"Ошибка: отсутствует ключ {e} в данных продукта '{name}'"
        except Exception as e:
            return f"Неизвестная ошибка при обработке продукта '{name}': {str(e)}"
    
    for res in result:
        result[res] =  result[res]/count * 100
    
    return result
        


@tool
def calculate_one_product_nutrition(product_name: str,count:int):
    """Посчитать общую ценность одного продукта в блюде"""
    if product_name in kk:
        result = {"squirrels_avg": kk[product_name]["squirrels_avg"]*count/100,
                "fats_avg": kk[product_name]["fats_avg"]*count/100,
                "carbohydrates_avg": kk[product_name]["carbohydrates_avg"]*count/100,
                "callories_avg": kk[product_name]["callories_avg"]*count/100,
                }
        return result
    return "Такого продукта нет в базе"
    			
    # Сумма белков/жиров/углеводов/калорий всех продуктов в корзине

In [48]:
# class MealPlanRequest(BaseModel):
#     """Параметры для составления плана питания"""
#     days: int = Field(description="Количество дней для плана питания", default=7)
#     dietary_preferences: list[str] = Field(description="Диетические предпочтения", default=[])
#     calories_per_day: float = Field(description="Целевое количество калорий в день", default=2000)

# @tool(args_schema=MealPlanRequest)
@tool
def create_weekly_meal_plan(days: int = 1, dietary_preferences = "", calories_per_day: float = 2000):
    """Составить план питания на указанное количество дней.
    
    Args:
        days: Количество дней для плана питания
        dietary_preferences: Предпочтения по продуктам
        calories_per_day: Целевое количество калорий в день
    """
    
    meal_plan = {}
    
    for day in range(1, days + 1):
        day_plan = {
            "breakfast": None,
            "lunch": None, 
            "dinner": None,
            "snacks": []
        }
        
        # Поиск блюд для каждого приема пищи
        breakfast_dishes = search_dish("завтрак"+ dietary_preferences)
        lunch_dishes = search_dish("обед"+ dietary_preferences)
        dinner_dishes = search_dish("ужин"+ dietary_preferences)
        
        if breakfast_dishes and len(breakfast_dishes) > 0:
            day_plan["breakfast"] = breakfast_dishes[0]  # Берем первое подходящее блюдо
        
        if lunch_dishes and len(lunch_dishes) > 0:
            day_plan["lunch"] = lunch_dishes[0]
            
        if dinner_dishes and len(dinner_dishes) > 0:
            day_plan["dinner"] = dinner_dishes[0]
        
        meal_plan[f"День {day}"] = day_plan
    
    return meal_plan

In [49]:
llm = ChatOpenAI(
    model="Qwen/Qwen3-235B-A22B-Instruct-2507",
    openai_api_key=os.environ.get("GIGA"),
    openai_api_base="https://foundation-models.api.cloud.ru/v1",
    max_tokens=2000
)




@wrap_tool_call
def handle_tool_errors(request, handler):
    """Обработка ошибок выполнения инструментов с пользовательскими сообщениями."""
    try:
        return handler(request)
    except Exception as e:
        # Возвращаем пользовательское сообщение об ошибке модели
        return ToolMessage(
            content=f"Ошибка инструмента: Пожалуйста, проверьте ваш ввод и попробуйте снова. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Генерация системного промпта на основе роли пользователя."""
    user_role = request.runtime.context.get("user_role", "user")
    

    if user_role == "expert":
        return f"{base_prompt} Предоставляйте подробные технические ответы."
    elif user_role == "beginner":
        return f"{base_prompt} Объясняйте концепции просто и избегайте профессионального жаргона."

    return base_prompt

agent = create_agent(
    llm,
    tools=[search_food,add_product_to_cart,show_product_in_cart,search_dish,remove_from_cart,clear_cart,
           calculate_all_products_nutrition,calculate_one_product_nutrition,create_weekly_meal_plan],
    middleware=[user_role_prompt,handle_tool_errors],
    context_schema=Context
)

# Системный промпт будет устанавливаться динамически на основе контекста


In [51]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Составь план питания на 2 дня, 2500 калорий, ветчина"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

К сожалению, возникла техническая ошибка при попытке сгенерировать план питания через инструмент. Однако я могу вручную составить для вас **сбалансированный план питания на 2 дня** с суточной калорийностью около **2500 ккал**, с включением **ветчины** как одного из основных продуктов.

---

## 🍽️ План питания на 2 дня | ~2500 ккал/день | С ветчиной

Цель:  
✅ Белок: ~110–130 г  
✅ Жиры: ~80–90 г  
✅ Углеводы: ~250–300 г  
✅ Акцент на ветчину, легкоусвояемые продукты, сбалансированность

---

### 📅 День 1

#### 🥣 Завтрак (650 ккал)
- Ветчина варёная — 100 г 🥩  
  (Б: 18 г, Ж: 6 г, У: 1 г, К: 140 ккал)
- Яйца варёные — 2 шт 🥚  
  (Б: 12 г, Ж: 10 г, У: 1 г, К: 140 ккал)
- Овсянка на воде — 60 г сушёной (готовой ~180 г) 🥣  
  (Б: 10 г, Ж: 3 г, У: 40 г, К: 250 ккал)
- Банан — 1 шт 🍌  
  (Б: 1 г, Ж: 0.5 г, У: 27 г, К: 110 ккал)
- Чай/кофе без сахара ☕

#### 🥪 Перекус (300 ккал)
- Ветчина — 50 г 🥩  
  (Б: 9 г, Ж: 3 г, У: 0.5 г, К: 70 ккал)
- Тост из цельнозернового хлеба — 2 ломтика 🍞  
  (Б:

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Сколько калорий будет в 400 грамм картофеля"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

Кажется, в базе данных отсутствует продукт под названием **"картофель"**. Однако, скорее всего, вы имели в виду **картофель в мундире**, **отварной картофель** или просто **картофель (сырой)** — такие варианты обычно присутствуют.

Давайте уточним! Вот приблизительная калорийность картофеля на основе общепринятых данных (средние значения на 100 г):

- **Сырой картофель**: ~77 ккал  
- **Отварной картофель (без соли, в кожуре)**: ~87 ккал  
- **Запечённый картофель**: ~93 ккал  
- **Жареный картофель (картофель фри)**: ~312 ккал (значительно выше из-за масла)

### Расчёт на 400 г отварного картофеля:
> 87 ккал × 4 = **348 ккал**

Если вы уточните способ приготовления (отварной, запечённый, жареный и т.д.), я смогу дать более точный ответ или найти продукт в базе под другим названием. 🥔💡

Хотите, я попробую поискать похожие продукты? Например: *картофель отварной*?


In [14]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Посоветуй блюдо,в котором есть сыр, картофель"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

Одно из вкусных и питательных блюд с **сыром** и **картофелем** — это **картофельные оладьи с сыром** 🧀🥔.

### 🍽️ Описание блюда:
- **Основные ингредиенты**: варёный картофель, твёрдый сыр.
- **Подача**: по 2 оладьи на порцию, подаются со сметаной или как гарнир.
- **Опционально**: могут подаваться с припущенными петушиными гребешками (для разнообразия вкуса и белка).

### ✅ Преимущества:
- Богато **белками** за счёт сыра.
- Умеренное количество **углеводов** от картофеля.
- Хорошо насыщает, подходит как для обеда, так и для ужина.

### 🔧 Как приготовить (простой вариант):
1. Отварите картофель в мундире, очистите и разомните в пюре.
2. Натрите твёрдый сыр на мелкой тёрке (например, 50 г на 200 г картофеля).
3. Смешайте картофель с сыром, добавьте яйцо, немного муки, соль, перец.
4. Сформируйте небольшие оладьи и обжарьте на сковороде до золотистой корочки.
5. Подавайте со сметаной или зеленью.

Хочешь, я помогу рассчитать калорийность одной порции или добавлю продукты в корзину для да

In [18]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Посоветуй 3 продукта, где больше 25 белков"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

Вот 3 продукта с содержанием белка более 25 г на 100 г продукта 🥩🥚🧀:

---

### 🌱 **Растительные источники белка**
1. **Соя (зерно)**  
   - 🥚 Белки: **36,7 г**  
   - 🧈 Жиры: 17,8 г  
   - 🍞 Углеводы: 17,3 г  
   - 🔥 Калории: 364 ккал  
   - ✅ Отличный растительный источник белка, подходит для веганов и вегетарианцев.

2. **Соевые бобы (варёные/свежие)**  
   - 🥚 Белки: **36,4 г**  
   - 🧈 Жиры: 19,9 г  
   - 🍞 Углеводы: 30,1 г  
   - 🔥 Калории: 446 ккал  
   - ✅ Богаты клетчаткой и изофлавонами, полезны для гормонального баланса.

---

### 🧀 **Животный белок**
3. **Сыр Гауда**  
   - 🥚 Белки: **25,0 г**  
   - 🧈 Жиры: 27,0 г  
   - 🍞 Углеводы: 2,0 г  
   - 🔥 Калории: 356 ккал  
   - ✅ Высококалорийный, но насыщенный белком и кальцием. Умеренное употребление — ключ.

---

### 💡 **Рекомендации:**
- Для **спорта и набора мышечной массы** — соя и соевые бобы — отличная основа рациона, особенно для веганов.
- **Гауда** — хороший выбор для перекуса или добавки в блюда, но следите за жирност

In [19]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Что можно приготовить с сыром?"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

🧀 **Что можно приготовить с сыром?**  
Сыр — универсальный продукт, который отлично подходит как для закусок, так и для горячих блюд. Вот подборка вкусных и простых рецептов, где сыр играет ключевую роль:

---

### 🥪 1. **Канапе с сыром**
- **Описание**: Классическая закуска на кусочке хлеба.
- **Ингредиенты**:
  - Хлеб (белый или багет)
  - Сыр (твердый, например, чеддер или гауда)
  - Масло сливочное
  - Дополнительно: ветчина, яйца, зелень, перец
- **Как готовить**:
  1. Намажь хлеб маслом.
  2. Положи ломтик сыра.
  3. Добавь ветчину и рубленые яйца (по желанию).
  4. Укрась зеленью и перцем.

💡 *Идеально для фуршета или лёгкого перекуса!*

---

### 🧈 2. **Гренки с сыром**
- **Описание**: Хрустящие тосты с расплавленным сыром.
- **Ингредиенты**:
  - Хлеб без корок
  - Тёртый сыр (лучше моцарелла или пармезан)
  - Сливочное масло
- **Как готовить**:
  1. Посыпь ломтики хлеба тёртым сыром.
  2. Сбрызни маслом.
  3. Запеки в духовке или тостере до золотистой корочки.

🔥 *Горячие, аром

In [20]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": 'Добавь в корзину Сыр "Гауда" (Гауда должен быть в таких кавыках), 500 грамм'}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

✅ Продукт **Сыр "Гауда"** — **500 грамм** успешно добавлен в корзину!  

🛒 **Текущее содержимое корзины:**  
- 🧀 Сыр "Гауда" — 500 г

Хочешь посмотреть полную питательную ценность этого продукта или добавить что-то ещё? 😊


In [21]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Напиши, какой вид хлеба есть в базе?"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

В базе данных представлено **12 видов хлеба и хлебцев** 🍞. Ниже — подробный список, сгруппированный по типам и с указанием ключевых питательных показателей на 100 г продукта.

---

### 🍞 **Традиционные сорта хлеба**

1. **Хлеб пшеничный из муки высшего сорта**  
   🟦 Белки: 7.55 г | 🟨 Жиры: 1.85 г | 🟩 Углеводы: 50.3 г | 🔥 Калории: 248.5 ккал  
   *Классический мягкий хлеб, высокое содержание углеводов.*

2. **Хлеб пшеничный из муки 1 сорта**  
   🟦 Белки: 7.6 г | 🟨 Жиры: 0.9 г | 🟩 Углеводы: 49.7 г | 🔥 Калории: 226.0 ккал  
   *Чуть менее калорийный, чем из высшего сорта муки.*

3. **Хлеб из ржано-пшеничной муки**  
   🟦 Белки: 6.8 г | 🟨 Жиры: 1.2 г | 🟩 Углеводы: 46.4 г | 🔥 Калории: 215.0 ккал  
   *Сбалансированный вкус, меньше углеводов.*

4. **Хлеб ржаной**  
   🟦 Белки: 4.7 г | 🟨 Жиры: 0.7 г | 🟩 Углеводы: 49.8 г | 🔥 Калории: 214.0 ккал  
   *Низкобелковый, но с характерным плотным вкусом.*

5. **Хлеб "Бородинский"**  
   🟦 Белки: 6.9 г | 🟨 Жиры: 1.3 г | 🟩 Углеводы: 40.9 г | 🔥 Калори

In [22]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Добавь в корзину Хлеб пшеничный из муки 1 сорта 300 грамм, Ветчина рубленая 400 грамм,Масло сливочное 300 грамм"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)


✅ Продукты успешно добавлены в корзину:

- 🍞 **Хлеб пшеничный из муки 1 сорта** — 300 г  
- 🥩 **Ветчина рубленая** — 400 г  
- 🧈 **Масло сливочное** — 300 г  

Хочешь посмотреть состав корзины или рассчитать общую питательную ценность этих продуктов? 📊


In [23]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Покажи, что в корзине"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

Вот что сейчас находится в вашей корзине 🛒:

🧀 **Сыр "Гауда"** — 500.0 г  
🍖 **Ветчина рубленая** — 400.0 г  
🍞 **Хлеб пшеничный из муки 1 сорта** — 300.0 г  
🧈 **Масло сливочное** — 300.0 г

Если хотите, могу рассчитать общую питательную ценность этих продуктов или помочь отредактировать корзину (добавить/удалить).


In [24]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Удали из корзины Хлеб пшеничный из муки 1 сорта, 200 грамм"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

Продукт **Хлеб пшеничный из муки 1 сорта** (200 грамм) успешно удалён из корзины. ✅

Если нужно — могу показать обновлённое содержимое корзины или помочь с другими изменениями. 🛒


In [25]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Удали из корзины Ветчина рубленая"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

Продукт **"Ветчина рубленая"** успешно удалён из корзины. ✅

Если нужно — могу показать текущее содержимое корзины или помочь добавить другие продукты! 🛒


In [26]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Покажи, что в корзине"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

Вот что сейчас находится в вашей корзине 🛒:

🧀 **Сыр "Гауда"** — 500.0 г  
🍞 **Хлеб пшеничный из муки 1 сорта** — 100.0 г  
🧈 **Масло сливочное** — 300.0 г

Это довольно калорийный набор с высоким содержанием жиров и углеводов. Если хотите, могу рассчитать общую питательную ценность этих продуктов или предложить более сбалансированный вариант.  
Нужна помощь с анализом или корректировкой? 😊


In [29]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Рассчитай общую питательную ценность продуктов в моей корзине"}]},
    context={"user_role": "expert"}
)
print(result['messages'][-1].content)

К сожалению, в базе данных только один вариант — **"Сыр \"Гауда\""** с кавычками, и система не принимает его из-за технических ограничений обработки спецсимволов.

### 🔧 Временное решение:
Я вручную рассчитаю питательную ценность всей корзины, используя данные из поиска.

---

## 🧮 Общая питательная ценность продуктов в корзине (ручной расчёт на 100 г → масштабируем)

### 📚 Источник данных:
- **Сыр "Гауда"** — 100 г: 25 г Б, 27 г Ж, 2 г У, 356 ккал  
- **Хлеб пшеничный из муки 1 сорта** — 100 г: 8.1 г Б, 1.2 г Ж, 48.7 г У, 258 ккал *(стандартные данные)*  
- **Масло сливочное** — 100 г: 0.6 г Б, 72.5 г Ж, 0.6 г У, 717 ккал *(стандартные данные)*

---

### 🔢 Расчёт по продуктам:

#### 1. 🧀 Сыр "Гауда" — 500 г
- Белки: 25 г × 5 = **125 г**
- Жиры: 27 г × 5 = **135 г**
- Углеводы: 2 г × 5 = **10 г**
- Калории: 356 ккал × 5 = **1780 ккал**

#### 2. 🍞 Хлеб пшеничный — 100 г
- Белки: **8.1 г**
- Жиры: **1.2 г**
- Углеводы: **48.7 г**
- Калории: **258 ккал**

#### 3. 🧈 Масло сливочное — 300 г